<a href="https://colab.research.google.com/github/PanoEvJ/LLMOps/blob/main/Week%203/Thursday/Deploy/Interacting_with_a_AWS_Hosted_Model_With_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
model_api_gateway = "https://e6g370y6pe.execute-api.us-east-1.amazonaws.com/default/Llama2API"

In [2]:
import requests

json_body = {
    "inputs" : [
       [{"role" : "system", "content" : "You are an expert in Python."},
       {"role" : "user", "content" : "How would I write a function to determine the Nth Fibonacci number?"}]
    ],
    "parameters" : {
        "max_new_tokens" : 256,
        "top_p" : 0.9,
        "temperature" : 0.1
    }
}

response = requests.post(model_api_gateway, json=json_body)

print(response.json())

[{'generation': {'role': 'assistant', 'content': " Great! I'd be happy to help you write a function to determine the Nth Fibonacci number in Python.\n\nThe Fibonacci sequence is a series of numbers where each number is the sum of the two preceding numbers, starting from 0 and 1. The sequence begins like this: 0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, and so on.\n\nTo write a function to determine the Nth Fibonacci number in Python, you can use the following approach:\n\n1. Define a function called `fibonacci` that takes an integer `n` as input.\n2. Use a recursive approach to calculate the `n`th Fibonacci number. The basic idea is to call the function with `n-1` and `n-2` as arguments, and then use the formula `fibonacci(n) = fibonacci(n-1) + fibonacci(n-2)`.\n3. Return the calculated value of `fibonacci(n)`.\n"}}]


In [3]:
!pip install -U -q langchain

In [4]:
from typing import Any, List, Mapping, Optional
from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM

class Llama2SageMaker(LLM):
    max_new_tokens: int = 256
    top_p: float = 0.9
    temperature: float = 0.1

    @property
    def _llm_type(self) -> str:
        return "Llama2SageMaker"

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
    ) -> str:
        if stop is not None:
            raise ValueError("stop kwargs are not permitted.")

        json_body = {
            "inputs" : [
              [{"role" : "user", "content" : prompt}]
            ],
            "parameters" : {
                "max_new_tokens" : self.max_new_tokens,
                "top_p" : self.top_p,
                "temperature" : self.temperature
            }
        }

        response = requests.post(model_api_gateway, json=json_body)

        return response.json()[0]["generation"]["content"]

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {
            "max_new_tokens" : self.max_new_tokens,
            "top_p" : self.top_p,
            "temperature" : self.temperature
        }

In [5]:
llm = Llama2SageMaker()

In [6]:
llm("How much wood could a woodchuck chuck if a wood chuck could chuck wood?")

" The answer to this tongue twister is a bit of a trick question! Woodchucks, also known as groundhogs, are not actually able to chuck wood. Woodchucks are burrowing animals that live in the ground and are known for their ability to dig tunnels and burrows. They do not have the ability to move or manipulate wood.\n\nSo, if a woodchuck could chuck wood, it would be able to move a very small amount of wood, perhaps a few small branches or twigs at a time. However, it's important to remember that woodchucks are not capable of chucking wood, so this is purely a hypothetical scenario!"

In [7]:
!pip install -q -U pymupdf

In [8]:
from langchain.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader("https://arxiv.org/pdf/2005.11401v4.pdf")

In [9]:
!pip install -q -U openai chromadb tiktoken

In [12]:
import getpass
import os

openai_api_key = getpass.getpass("Please provide your OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = openai_api_key

Please provide your OpenAI API Key: ··········


In [13]:
from langchain.indexes import VectorstoreIndexCreator

index = VectorstoreIndexCreator().from_loaders([loader])

In [14]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm, retriever=index.vectorstore.as_retriever(), return_source_documents=True
)

In [15]:
question = "What is Retrieval Augmented Generation?"
result = qa_chain({"query": question})
result["result"]

' Based on the provided context, Retrieval Augmented Generation (RAG) is a technique that combines the strengths of retrieval-based models and generation models to improve the performance of knowledge-intensive NLP tasks. RAG models use a retrieval component to access and manipulate knowledge in a non-parametric manner, and a generation component to generate text that is factual and specific. The goal of RAG is to unify previous successes in incorporating retrieval into individual tasks and show that a single retrieval-based architecture can achieve strong performance across several tasks.'

In [16]:
question = "What is Retrieval Augmented Generation?"
result = qa_chain({"query": question})
result

{'query': 'What is Retrieval Augmented Generation?',
 'result': ' Based on the provided context, Retrieval Augmented Generation (RAG) is a technique that combines the strengths of retrieval-based models and generation models to improve the performance of knowledge-intensive NLP tasks. RAG models use a combination of parametric and non-parametric memory to generate high-quality text, and they have shown promising results in open-domain question answering tasks. The key idea behind RAG is to leverage the strengths of retrieval-based models, which are good at retrieving relevant information, and generation models, which are good at generating coherent and fluent text. By combining these two approaches, RAG models can generate more informative and accurate text than either approach alone.',
 'source_documents': [Document(page_content='including less of emphasis on lightly editing a retrieved item, but on aggregating content from several\npieces of retrieved content, as well as learning lat